# Validador automático erros CAR x POS - Cupons

Autor Danilo Fornari - dfornari@icloud.com

In [ ]:
#importando pacotes
import os
import json
import xlsxwriter
from shutil import copyfile
import pandas as pd
import numpy as np
import csv
from datetime import datetime
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")
import requests

data_set = {'items':[]}
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('SAP CAR Validador\nStarting Operation\n')
print('PLEASE MAKE SURE THAT THE LOADED FILES ARE AS FOLLOWS:\
\nSAP.CSV ->;<- separated\nSAP_HEAD.CSV ->;<- separated\nLIVE.CSV ->,<- separated \n \
site.CSV ->;<- separated\n')
print('************************************************************************\n')
#definindo diretorios de trabalho
path = os.getcwd()
#diretorio de export de resultados
path2 = path+"/EXPORT"
if not os.path.exists(path2):
    os.makedirs(path2)

#cabeçalho do arquivo DETAIL_REPORT.CSV output
detail_report_header=['CNPJ','SAP_ID','DATA','CUPOM','STATUS','LIVE_PROTOCOLO','184_QTD_CUPONS', \
                      '184_QDE_SKU','184_AMOUNT']

#criando DETAIL_REPORT.CSV output 
detail_report_df = pd.DataFrame(columns = detail_report_header)

#Opening SAP.CSV
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Opening SAP.CSV file\n')
print('************************************************************************\n')
sapdf = pd.read_csv('SAP.csv', sep = ";", engine = 'python')
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Working on SAP.CSV file\n')
print('************************************************************************\n')
#adding CNPJ Column
sapdf['CNPJ'] =  sapdf['Sold-To Party']
#importing SAPCODE->CNPJ Table
sitedf = pd.read_csv('site.csv', sep = ';', engine='python')
code_to_cnpj = {}
for i in sitedf.values:
    code_to_cnpj[i[0]]=i[1]
#doing the map to CNPJ column
sapdf['CNPJ'] =  sapdf['CNPJ'].map(code_to_cnpj)
sapdf['CNPJ'] = sapdf['CNPJ'].astype(str)
#Creating column Cupon from Transaction Number
sapdf['Coupon']=sapdf['Transaction Number'].apply(str)
#Slicing to remove date from cupon number 
sapdf['Coupon'] = sapdf['Coupon'].str[8:]
#Striping to remove left zeros 
sapdf['Coupon'] = sapdf['Coupon'].apply(lambda x: x.lstrip('0'))
#Removing . for number above 1000
sapdf['Amount(POSDM)'] = sapdf['Amount(POSDM)'].apply(lambda h: h.replace(".",""))
#Creating column UUID for VLOOKUP
sapdf['UUID']=sapdf['CNPJ'].astype(str)+sapdf['Coupon']+sapdf['Billing Date'].astype(str)+sapdf['Amount(POSDM)'].astype(str)


#Removing right zeros and ',from UUID of products
sapdf['UUID'] = \
sapdf['UUID'].astype(str).apply(lambda x: x.rstrip('0'))
sapdf['UUID'] = \
sapdf['UUID'].astype(str).apply(lambda x: x.rstrip(','))



#SAP.CSV changes are finished
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Work Finished on SAP.CSV file\n')
print('************************************************************************\n')

#Opening LIVE.CSV
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Opening LIVE.CSV file\n')
print('************************************************************************\n')
livedf = pd.read_csv('LIVE.csv', sep = ",", engine = 'c', encoding = 'utf_16', skiprows =4 )
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Working on LIVE.CSV file\n')
print('************************************************************************\n')
#Convertinnng CNPJ in string
livedf['CNPJ_LOJA'] = livedf['CNPJ_LOJA'].apply(str)
#Creating column Data with sliced data DD/MM/YY
livedf['data'] = livedf['DATA_DOCUMENTO'].str[0:8]
livedf['data'] = livedf['data'].astype(str)
#Creating value with sliced data from VALOR_PAGO
livedf['value']= livedf['VALOR_PAGO'].str[3:]
#Creating column UUID for VLOOKUP
livedf['UUID']=livedf['CNPJ_LOJA'].astype(str)+livedf['NUMERO_DOCUMENTO'].astype(str)\
+livedf['data'].astype(str)+livedf['value']
#Deleting Cancelled coupons
livedf.drop(livedf[livedf['SITUACAO_DOCUMENTO']=='CANCELADO'].index, inplace=True)






#Logging and deleting missing payment - V4 22/06
if livedf['value'].isnull().any() == True:
    #Creatinng DF with no payment coupond
    missingcpdf = livedf[livedf['value'].isnull()]
    #Logging info in detail_report_df
    for misslin in  missingcpdf.values:
        misslin2 = [misslin[5],np.nan,misslin[4][0:8],misslin[9],'Missing Payment LIVE',str(misslin[1]),np.nan,np.nan,np.nan]
        detail_report_df.loc[len(detail_report_df)] = misslin2
    #Apagando os duplicados
    livedf.drop(livedf[livedf['value'].isnull()].index, inplace = True)
#Showing Duplicated values
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Showing Duplicated lines:\n \n')
print(livedf[(livedf.duplicated(subset = 'UUID'))==True])
print('************************************************************************\n')
#Deleting UUID Duplicated Values
livedf.drop_duplicates(subset='UUID',inplace=True)
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Work Finished on LIVE.CSV file\n')
print('************************************************************************\n')

#Opening SAP_HEAD.CSV
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Opening SAP_HEAD.CSV file\n')
print('************************************************************************\n')
sap_headdf = pd.read_csv('SAP_HEAD.csv', sep = ";", engine = 'python')
#dropping nnull lines
sap_headdf.dropna(subset=['store ID'], inplace = True)
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Working on SAP_HEAD.CSV file\n')
print('************************************************************************\n')
#adding CNPJ Column
sap_headdf['CNPJ'] =  sap_headdf['store ID']
#doing the map to CNPJ column
sap_headdf['CNPJ'] =  sap_headdf['CNPJ'].map(code_to_cnpj)
#Removing right zeros and '.' from quantity of coupons - Removed on V3
#sap_headdf['Receipts of statistical(POS)'] = \
#sap_headdf['Receipts of statistical(POS)'].astype(str).apply(lambda x: x.rstrip('0'))
#sap_headdf['Receipts of statistical(POS)'] = \
#sap_headdf['Receipts of statistical(POS)'].astype(str).apply(lambda x: x.rstrip('.'))
#Removing thousand '.' point from totam amout
sap_headdf['Amount(POS)'] = \
sap_headdf['Amount(POS)'].apply(lambda x: x.replace(".",""))
#Removing right zeros and ','rom quantity of products
sap_headdf['Quantity(POS)'] = \
sap_headdf['Quantity(POS)'].astype(str).apply(lambda x: x.rstrip('0'))
sap_headdf['Quantity(POS)'] = \
sap_headdf['Quantity(POS)'].astype(str).apply(lambda x: x.rstrip(','))
#Creating control/sumarise coluns
sap_headdf['Live_184_Error'] = np.nan
sap_headdf['Live_Missing_Coupon'] = np.nan
sap_headdf['SAP_Cancelled_Error'] = np.nan
sap_headdf['SAP_ReturnNFE_Error'] = np.nan
sap_headdf['SAP_Duplicated_Error'] = np.nan
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Work Finished on SAP_HEAD.CSV file\n')
print('************************************************************************\n')

#Creating a set o variables that will be used as subset filters to the datasets
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Creating program variables \n')
print('************************************************************************\n')
cnpj = str
data_venda = str
numero_cupons_sap = str
valor_total_sap = str
quantidade_sap = str
numero_cupons_live = str
valor_total_live = str
quantidade_live = str
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Program variables successfully created \n')
print('************************************************************************\n')

#BEGINNING OF THE ANALISYS
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Beginning of Analisys \n')
print('************************************************************************\n')
#Opening FOR Loop reading lines of SAP Head
for sapheadline in sap_headdf.index:
    #line index is i-1
    line=0
    line = sapheadline-1
    #attributing variables values 
    cnpj = str(sap_headdf.iloc[line]['CNPJ'])
    data_venda = str(sap_headdf.iloc[line]['Sales date'])
    numero_cupons_sap = sap_headdf.iloc[line]['Receipts of statistical(POS)']
    if numero_cupons_sap == '':
        numero_cupons_sap = 0
    valor_total_sap = sap_headdf.iloc[line]['Amount(POS)']
    quantidade_sap = sap_headdf.iloc[line]['Quantity(POS)']
    
    #Creating subsets
    print('************************************************************************\n')
    print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
    print('Creating subsets for SAP HEAD line:' , line, '\n')
    print('Saving Variables for SAP HEAD line:' , line, '\n')
    print('************************************************************************\n')
    #creating LIVE_AN subset
    livedf_an = livedf[(livedf['data']==data_venda) & (livedf['CNPJ_LOJA']==cnpj)]

    #Checking cupons missing payment in LIVE ---V3
    storecode1 = str
    storecode1 = str(sap_headdf.iloc[[line],[0]].values[0,0])
    if livedf_an['value'].isnull().any() == True:
        #Creatinng DF with no payment coupond
        missingcpdf = livedf_an[livedf_an['value'].isnull()]
        #Logging info in detail_report_df
        for misslin in  missingcpdf.values:
            misslin2 = [misslin[5],storecode1,misslin[4][0:8],misslin[9],'Missing Payment LIVE',str(misslin[1]),np.nan,np.nan,np.nan]
            detail_report_df.loc[len(detail_report_df)] = misslin2
        #Apagando os duplicados
        livedf_an.drop(livedf_an[livedf_an['value'].isnull()].index, inplace = True)

    
    #adjusting data type of value column
    livedf_an['value'] = livedf_an['value'].apply(lambda h: h.replace(",","."))
    livedf_an['value'] = livedf_an['value'].astype(float)
    #creating SAP_AN subset
    sapdf_an = sapdf[(sapdf['Billing Date']==data_venda) & (sapdf['CNPJ']==cnpj)]
    sapdf_an['Amount(POSDM)'] = sapdf_an['Amount(POSDM)'].apply(lambda h: h.replace(",","."))
    sapdf_an['Amount(POSDM)'] = sapdf_an['Amount(POSDM)'].astype(float)
    #Removing right zeros and ','rom quantity of products
    sapdf_an['Quantity(POSDM)'] = \
    sapdf_an['Quantity(POSDM)'].astype(str).apply(lambda x: x.rstrip('0'))
    sapdf_an['Quantity(POSDM)'] = \
    sapdf_an['Quantity(POSDM)'].astype(str).apply(lambda x: x.rstrip(','))
    sapdf_an['Quantity(POSDM)'] = sapdf_an['Quantity(POSDM)'].astype(int)
    #Exporting DF for further anlisys
    sapdf_an.to_excel('EXPORT/'+str(line)+'SAPDF.xlsx',engine='xlsxwriter')
    livedf_an.to_excel('EXPORT/'+str(line)+'LIVEDF.xlsx',engine='xlsxwriter')
    
    
    #Checking for duplicated values inn SAP_AN
    if (sapdf_an.duplicated(subset='UUID')).any() == True:
        print('************************************************************************\n')
        print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
        print('SAP Duplicated coupons have been found \n')
        print('************************************************************************\n')
        duplicateddf = sapdf_an[sapdf_an.duplicated(subset='UUID')].copy()
        newdupliline =[]
        #Logging info in detail_report_df
        for duplilin in  duplicateddf.values:
            newdupliline = [duplilin[10],duplilin[0],duplilin[2],duplilin[-2],'Duplicated',np.nan,np.nan,np.nan,np.nan]
            detail_report_df.loc[len(detail_report_df)] = newdupliline
        #Logging info in sap_headdf
        sap_headdf.iloc[[line],[-3]]='X'
        #Deleting UUID Duplicated Values
        sapdf_an.drop_duplicates(subset='UUID',inplace=True)
        
    #Checking for LIVE NFE values inn SAP_AN
    if (sapdf_an['POS Number']=='LIVE NFE').any() == True:
        print('************************************************************************\n')
        print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
        print('SAP LIVE NFE coupons have been found \n')
        print('************************************************************************\n')
        livenfedf = sapdf_an[sapdf_an['POS Number']=='LIVE NFE'].copy()
        newlivenfelin = []
        #Logging info in detail_report_df
        for livenfelin in  livenfedf.values:
            newlivenfelin = [livenfelin[10],livenfelin[0],livenfelin[2],livenfelin[-2],'LIVE NFE',np.nan,np.nan,np.nan,np.nan]
            detail_report_df.loc[len(detail_report_df)] = newlivenfelin
        #Logging info in sap_headdf
        sap_headdf.iloc[[line],[-2]]= 'X'
        #Deleting LIVE NFE Values
        sapdf_an.drop(sapdf_an[sapdf_an['POS Number']=='LIVE NFE'].index,inplace=True)
        
    #Checking for Cancelled values in SAP_AN
    if sapdf_an['Transaction Number'].str.contains('C').any() == True:
        print('************************************************************************\n')
        print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
        print('SAP Cancelled Coupons have been found \n')
        print('************************************************************************\n')
        cancdf = sapdf_an[sapdf_an['Transaction Number'].str.contains('C')].copy()
        canclinlin = []
        cancdrop = []
        #Logging info in detail_report_df
        for canclin in  cancdf.values:
            canclinlin = [canclin[10],canclin[0],canclin[2],canclin[-2],'Cancelled',np.nan,np.nan,np.nan,np.nan]
            cancdrop.append(canclin[7][0:-1])
            detail_report_df.loc[len(detail_report_df)] = canclinlin
        #Logging info in sap_headdf
        sap_headdf.iloc[[line],[-1]]='X'
        #Deleting CANC and ACTIVE_cancelled Coupons
        sapdf_an.drop(sapdf_an[sapdf_an['Transaction Number'].str.contains('C')].index,inplace=True)
        for dropcancvalues in cancdrop:
            sapdf_an.drop(sapdf_an[sapdf_an['Transaction Number']==dropcancvalues].index,inplace=True)
            
    #Matching variables to see if need to go further
    #Validating if the Total Amout is between 2 and -2
    valor_total_sap_an=sapdf_an['Amount(POSDM)'].sum()
    valor_total_sap_float = float(valor_total_sap.replace(',','.'))
    valid_amount = (valor_total_sap_an - valor_total_sap_float)<2 and (valor_total_sap_an - valor_total_sap_float)>-2
    #Validating if the Total number of coupons maches
    numero_cupons_sap_an = sapdf_an['Amount(POSDM)'].count()
    numero_cupons_sap_int = int(numero_cupons_sap)
    valid_amount_coupons = (numero_cupons_sap_int==numero_cupons_sap_an)
    #Validating if the Total number of products maches
    quantidade_sap_an = sapdf_an['Quantity(POSDM)'].sum()
    quantidade_sap_int = int(quantidade_sap)
    valid_amout_products = (quantidade_sap_int == quantidade_sap_an)
    #if all validation keys are true so there are no more divergences to check in this line
    #if any validation key is false them we need to move forward to cross checking with LIVE
    if valid_amount==True and valid_amount_coupons==True and valid_amout_products==True:
        print('************************************************************************\n')
        print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
        print('SAP Cleasing cleared all divergences in line:', line, ' hence moving to line: ' , line+1, '\n')
        print('************************************************************************\n')
    else:
        print('************************************************************************\n')
        print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
        print('SAP Cleasing was did not resolved all the divergences, hence moving forward to LIVE cross checking\n')
        print('************************************************************************\n')
        #Doing the Join and saving this in joineddf 
        storecode = str
        storecode = str(sap_headdf.iloc[[line],[0]].values[0,0])
        joinedf = livedf_an.join(sapdf_an.set_index('UUID'), on='UUID')
        #Checking whether there is any missing coupon
        if joinedf['Transaction Number'].isnull().any() == True:
            print('************************************************************************\n')
            print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
            print('Missing coupons have been found in VLOOKUP UUID \n')
            print('************************************************************************\n')
            #Separating missing coupons
            missingcpdf = joinedf[joinedf['Transaction Number'].isnull()]
            #Logging info in detail_report_df
            for misslin in  missingcpdf.values:
                misslin2 = [misslin[5],storecode,misslin[4][0:8],misslin[9],'Missing Coupon',str(misslin[1]),np.nan,np.nan,np.nan]
                detail_report_df.loc[len(detail_report_df)] = misslin2
            #Logging info in sap_headdf
            sap_headdf.iloc[[line],[-4]] = 'X'
        else:
            print('************************************************************************\n')
            print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
            print(' *NO* Missing coupons have been found in VLOOKUP UUID \n')
            print('************************************************************************\n')
        #Matching variables to see if need to go further
        #Validating if the Total Amout is between 2 and -2
        valor_total_live_an=livedf_an['value'].sum()
        valid_amount_live = (valor_total_live_an - valor_total_sap_float)<2 and (valor_total_live_an - valor_total_sap_float)>-2    
        #Validating if the Total number of coupons maches
        numero_cupons_live_an = livedf_an['value'].count()
        valid_amount_coupons_live = (numero_cupons_sap_int==numero_cupons_live_an)
        if valid_amount_live==True and valid_amount_coupons_live==True:
            print('************************************************************************\n')
            print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
            print('*NO* 184 Errors Founded in line:', line, ' hence moving to line: ' , line+1, '\n')
            print('************************************************************************\n')
        else:
            print('************************************************************************\n')
            print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
            print('184 Errors Founded in line:', line, '\n')
            print('************************************************************************\n')
            #Logging info in sap_headdf
            sap_headdf.iloc[[line],[-5]] = 'X'
            #Logging info in detail_report_df
            new184felin = [cnpj,storecode,data_venda,np.nan,'Reprocess 184',np.nan,numero_cupons_sap_an,quantidade_sap_an,valor_total_sap_an]
            detail_report_df.loc[len(detail_report_df)] = new184felin
            #Appending 184 data to compose Json output
            #Creatinng the variables Value as float and date as string
            #value_184=valor_total_sap.replace(',','.')
            data_184=sap_headdf.iloc[[line],[-7]].values
            data_set['items'].append\
            ({"fkdat": str(int(data_184)),"fkimg": int(quantidade_sap_an),"brtwr": round(valor_total_sap_an,2),"count_pos": int(numero_cupons_sap_an),\
              "werks": str(storecode),"waerk": "BRL"})
            
            
#exporting detail_report_df to excel
detail_report_df['LIVE_PROTOCOLO'] = detail_report_df['LIVE_PROTOCOLO'].astype(str)
detail_report_df.to_excel('EXPORT/#Detail_report_FINAL.xlsx',engine='xlsxwriter')
#exporting sap_headdf to excel
sap_headdf['CNPJ'] = sap_headdf['CNPJ'].astype(str)
sap_headdf.to_excel('EXPORT/#Sap_Head_FINAL.xlsx',engine='xlsxwriter')
#exporting Json file
with open('EXPORT/184_message.json', 'w') as outfile:
    json.dump(data_set, outfile)
#sending json payload to SAP URL automatically
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Posting 184 ajustments to SAP''\n')
print('************************************************************************\n')  
url = "https://" #PUT YOUR ENDPOINT HERE
payload = json.dumps(data_set)
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic' #PUT YOUR KEY HERE
}
response = requests.request("POST", url, headers=headers, data = payload)
print('\nSAP Response: ',response )
print(response.text.encode('utf8'))

print('\n\n************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('----------->END<-----------\n')
print('************************************************************************\n')

#V5
#Automatic reprocessinng coupons
print('************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('Automatic reprocessinng coupons to SAP''\n')
print('************************************************************************\n') 

print('Creating DF''\n')

sd022df = detail_report_df[detail_report_df['STATUS']=='Missing Coupon']
sd022df.reset_index(drop=True,inplace=True)
sd022df['Sent_Status']= 'no'

#Preparing the payload for each line and sending this line by line
for list_line in sd022df.index:
    json_sd022 = {"numeroTicket": str(sd022df.iloc[list_line]['LIVE_PROTOCOLO']),\
              "listaFiltro":[{"numeroCupom":str(sd022df.iloc[list_line]['CUPOM']),"cnpj": str(sd022df.iloc[list_line]['CNPJ'])}]}
    payload = json.dumps(json_sd022)
    print('************************************************************************\n')
    print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
    print('Sending line:', list_line, ' Payload:\n')
    print(payload)
    print('************************************************************************\n')
    
    #Sending data to API
    url = "https://" #PUT YOUR ENDPOINT HERE
    headers = {'Content-Type': 'application/json'}
    response = requests.request("POST", url, headers=headers, data = payload)
    print(response.text.encode('utf8'))
    if (response.text.find("success") != -1):
        sd022df.iloc[[list_line],[-1]] = 'Yes'

sd022df.to_excel('EXPORT/Coupon_Reprocessing_Report.xlsx',engine='xlsxwriter')

print('\n\n************************************************************************\n')
print("Current Time =", datetime.now().strftime("%H:%M:%S"),'\n')
print('----------->END<-----------\n')
print('************************************************************************\n')